<a href="https://colab.research.google.com/github/zlximon/Tensorflow_DL/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#downloading helper functions 
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2022-07-12 06:23:53--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-12 06:23:53 (99.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys



#get text dataset

In [3]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-07-12 06:23:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.128, 142.251.12.128, 172.217.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-07-12 06:23:56 (98.2 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



  #visualizing text dataset

In [4]:
import pandas as pd 

In [5]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()
# 1 is for disaster 

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
len(train_df),len(test_df)

(7613, 3263)

In [10]:
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
That day remember the nuclear weapon power......Hiroshima 70th

---

Target: 0 (not real disaster)
Text:
@nuggets #FETTILOOTCH IS #SLANGLUCCI OPPRESSIONS GREATEST DANGER COMING SOON THE ALBUM 
https://t.co/moLL5vd8yD

---

Target: 0 (not real disaster)
Text:
Gov. Brown allows parole for 1976 Chowchilla school bus hijacker James Schoenfeld. http://t.co/GlWoAKppjq http://t.co/WlLfLFnDgG

---

Target: 0 (not real disaster)
Text:
@justinbieber   I AM SCREAMING HELL YES AHHHH OMG http://t.co/y678XsNvJ6

---

Target: 1 (real disaster)
Text:
Ted Cruz Bashes Obama Comparison GOP To Iranians Shouting 'Death To America' http://t.co/tXETcysm1H | #tcot

---



#split data into training and validation  sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),train_df_shuffled["target"],test_size=0.1,
                                                                       random_state=42)

In [13]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [14]:
train_sentences[:10],train_labels[:10 ]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), 5921    0


Embedding : create a matrix of features vector for each token 

#text vectorization ( tokenization)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [16]:
text_vectorizer=TextVectorization(max_tokens=None# how many word in vocalubary  and None means there is no cap 
                                  ,standardize="lower_and_strip_punctuation",# turn all words to lower case and strip 
                                  split="whitespace",ngrams=None,output_mode="int",output_sequence_length=None,# how long do we want sequences be 
                                 )# pad_to_max_tokens=True)#add zeros into end of short lenght 

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None # how long do you want your sequences to be?
                                    )

In [18]:
#find the average numbers of tokens
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [19]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [20]:
# fit the text vectorization to the training text
text_vectorizer.adapt(train_sentences)


In [21]:
#create a sample 
sample="There is a flood in my street"
text_vectorizer([sample])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
#choose a random sentence from training dataset
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Second tornado confirmed in SundayÛªs storm http://t.co/Ffnzit7kgN        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 642,  496,  280,    4, 7936,   84,    1,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # the most common words in the vocab
bottom_5_words = words_in_vocab[-5:] # the least common words in the vocab
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab: {bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#creating Embedding layer 

Embedding : turns positives integers into dense vectors of fixed size
parameters for our embedding

1-input_dim - The size of the vocabulary

2-output_dim - The size of the output embedding vector, for example, a value of 100 outputs a feature vector of size 100 for each word.

3-embeddings_initializer - How to initialize the embeddings matrix, default is "uniform" which randomly initalizes embedding matrix with uniform distribution. This can be changed for using pre-learned embeddings.

4- input_length - Length of sequences being passed to embedding layer.


In [24]:
from tensorflow.keras import layers 
embedding =layers.Embedding(input_dim=max_vocab_length,output_dim=128,input_length=max_length)
embedding

In [25]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
WHY THE DEEP ROADS THO HAHAHAHA IM SO TRAUMATISED BY THE DEEP ROADS LOLOL      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03872079, -0.03897832,  0.04987678, ...,  0.00430049,
          0.0014724 , -0.04110668],
        [-0.01772659,  0.04298625, -0.04918823, ...,  0.03742156,
          0.02959874,  0.01109803],
        [ 0.04534731, -0.04177235, -0.02712531, ..., -0.03610072,
         -0.029843  , -0.00139116],
        ...,
        [ 0.00137011, -0.00166489, -0.02641318, ..., -0.01447127,
         -0.04997814,  0.04486228],
        [ 0.02703425,  0.00787104, -0.02891326, ..., -0.0249662 ,
          0.00184073, -0.00633626],
        [ 0.02989021, -0.04672978, -0.04140582, ..., -0.03993809,
          0.01318102, -0.04572009]]], dtype=float32)>

In [26]:
sample_embed[0][0],sample_embed[0][0].shape,random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 3.87207903e-02, -3.89783159e-02,  4.98767756e-02, -2.99100876e-02,
         4.88669612e-02, -3.89848948e-02, -1.26337782e-02, -2.42993236e-02,
         3.28510292e-02,  2.07145140e-03,  2.03397386e-02, -9.43051651e-03,
        -3.42552662e-02,  2.85014249e-02,  4.13781144e-02, -4.38897721e-02,
        -7.99428672e-04, -4.21239845e-02,  2.15727091e-03,  2.87311785e-02,
        -1.57035887e-04, -2.30744481e-02, -5.20860031e-03,  3.17147635e-02,
        -3.56592312e-02, -1.31921768e-02,  3.75108607e-02,  5.22360206e-05,
         1.92322247e-02, -1.49556026e-02, -1.61442049e-02,  1.07730515e-02,
         1.74438618e-02,  2.39568464e-02, -2.05135942e-02,  3.59847881e-02,
        -3.17217596e-02, -3.46598513e-02, -3.47204134e-03,  1.35538913e-02,
        -1.94611400e-03,  1.62834562e-02, -4.69408743e-02,  2.39259712e-02,
         1.40820183e-02, -3.50461155e-03, -6.51679933e-04, -4.40915935e-02,
         4.05522622e-02,  2.58335136e-0

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


In [28]:
model0=Pipeline([
("tfidf",TfidfVectorizer()),#convert words to numbers using tfdif
("clf",MultinomialNB())#model the text
])
#fit pipeline to train data
model0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
baseline_score=model0.score(val_sentences,val_labels)# evaluate 
baseline_score # accuracy 

0.7926509186351706

In [30]:
baseline_pred=model0.predict(val_sentences)
baseline_pred[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

#creating function to track and evaluate model 

In [31]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  ----
  y_true = true labels in the form of a 1D array
  y_pred = predicted label in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall and f1-score between y_true and y_pred.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100 # get accuracy score in percentage value
  # Calculate model precision, recall and f1 score using "weighted" avergage
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  # Create a dictionary of model results
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [32]:
baseline_result=calculate_results(y_true=val_labels,y_pred=baseline_pred)
baseline_result

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

#model 1 (Dense model)

In [33]:
from helper_functions import create_tensorboard_callback


In [34]:
SAVE_DIR = "model_logs"


In [35]:
from tensorflow.keras import layers
inputs=(layers.Input(shape=(1,),dtype=tf.string))
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
#x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model1=tf.keras.Model(inputs,outputs,name="model1")

In [36]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [37]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [38]:
#compile model
model1.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])


In [39]:
#fit the model
model1_history=model1.fit(x=train_sentences,y=train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20220712-062405
Epoch 1/5
215/215 [==============================] - 9s 10ms/step - loss: 0.6094 - accuracy: 0.7025 - val_loss: 0.5349 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4417 - accuracy: 0.8199 - val_loss: 0.4714 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3474 - accuracy: 0.8606 - val_loss: 0.4610 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2844 - accuracy: 0.8888 - val_loss: 0.4640 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2374 - accuracy: 0.9133 - val_loss: 0.4783 - val_accuracy: 0.7795


In [40]:
model1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4783 - accuracy: 0.7795


[0.4783119261264801, 0.7795275449752808]

In [41]:
model1_pred_probs=model1.predict(val_sentences)
model1_pred_probs.shape

(762, 1)

In [42]:
model1_pred_probs[0]

array([0.4447921], dtype=float32)

In [43]:
#Convert model prediction probabilites to label format 
model1_preds=tf.squeeze(tf.round(model1_pred_probs))
model1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [44]:
model1_result=calculate_results(y_true=val_labels,y_pred=model1_preds)
model1_result


{'accuracy': 77.95275590551181,
 'f1': 0.7771404562571971,
 'precision': 0.7822644211580037,
 'recall': 0.7795275590551181}

In [45]:
baseline_result

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

#visualazing learned embeddings

In [46]:
#get the vocabulary from  the vectorization layer
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [47]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [48]:
embed_weights=model1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [49]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [50]:
# Let's download the saved embeddings locally

try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#recurrent neural Networks (RNN)

#model 2 LSTM 
long short term momory 

In [51]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
#x=layers.LSTM(64,return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) 
x=layers.Dense(64,activation="relu")(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model2=tf.keras.Model(inputs,outputs,name="model2")

In [52]:
model2.summary()

Model: "model2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                            

In [53]:
#compile the model
model2.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [54]:
#fit the model
model2_history=model2.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels), callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220712-062429
Epoch 1/5
215/215 [==============================] - 7s 9ms/step - loss: 0.2145 - accuracy: 0.9218 - val_loss: 0.6205 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1542 - accuracy: 0.9412 - val_loss: 0.6534 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1258 - accuracy: 0.9495 - val_loss: 0.7101 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1012 - accuracy: 0.9609 - val_loss: 0.7224 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0808 - accuracy: 0.9663 - val_loss: 1.0220 - val_accuracy: 0.7730


In [55]:
#make prediction 
model2_pred_probs=model2.predict(val_sentences)
model2_pred_probs[:10]

array([[9.15523712e-03],
       [6.93078518e-01],
       [9.99978900e-01],
       [6.67761192e-02],
       [1.01034995e-04],
       [9.99857306e-01],
       [9.54206526e-01],
       [9.99994278e-01],
       [9.99985695e-01],
       [4.26873356e-01]], dtype=float32)

In [56]:
# Round out pred probs and reduce to 1-dimensional array

model_2_preds = tf.squeeze(tf.round(model2_pred_probs))


In [57]:
# Calculate our LSTM model's results

model_2_preds = tf.squeeze(tf.round(model_2_preds))


In [58]:
# Calculate our model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.29658792650919,
 'f1': 0.7710949612836401,
 'precision': 0.7742755789705,
 'recall': 0.7729658792650919}

#model 3 - GRU model

In [59]:
#build RNN model using GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x=layers.LSTM(64,return_sequences=True)(x)
#x=layers.GRU(64,)(x)
#x=layers.Dense(64,activation="relu")(x)
x=layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")


In [60]:
#model 3 summary with out GlobalAverage pooling
model3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non

In [61]:
#model 3 summary with our GlobalAverage pooling
model3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non

In [62]:
#compile the model
model3.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [63]:
#fit the model
model3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220712-062452
Epoch 1/5
215/215 [==============================] - 2s 5ms/step - loss: 0.5155 - accuracy: 0.8175 - val_loss: 0.5009 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3196 - accuracy: 0.9038 - val_loss: 0.4593 - val_accuracy: 0.7900
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2403 - accuracy: 0.9228 - val_loss: 0.4640 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1968 - accuracy: 0.9378 - val_loss: 0.4882 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1666 - accuracy: 0.9495 - val_loss: 0.5069 - val_accuracy: 0.7822


In [64]:
model3_pred_probs=model3.predict(val_sentences)
model3_pred_probs[:10]

array([[0.4118144 ],
       [0.7971384 ],
       [0.9973149 ],
       [0.10236871],
       [0.03937065],
       [0.9429246 ],
       [0.8845228 ],
       [0.9972114 ],
       [0.99108815],
       [0.3086877 ]], dtype=float32)

In [65]:
model3_preds=tf.squeeze(tf.round(model3_pred_probs))
model3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [66]:
model3_result=calculate_results(y_true=val_labels,y_pred=model3_preds)
model3_result

{'accuracy': 78.21522309711287,
 'f1': 0.7797935460636591,
 'precision': 0.7849731976577976,
 'recall': 0.7821522309711286}

#Model4 bidirectional RNN
normal RNN's go from left to right 
but bidirectional RNN goes from right to left as well as left to right

In [67]:
#build a model with tensorflow 
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
#x=layers.Bidirectional(layers.GRU(64,return_sequences=True))(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model4=tf.keras.Model(inputs,outputs,name="model4_bidirectional")

                                                             

In [68]:
#our model summary
model4.summary()
#the shape is 128 (instead of 64 ) because we are going in 2direcitons (so it dobules the value )

Model: "model4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,37

In [69]:
#compile the model
model4.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [70]:
# fit the model
model4.history=model4.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels),
                          callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_4")])

Saving TensorBoard log files to: model_logs/model_4/20220712-062458
Epoch 1/5
215/215 [==============================] - 6s 13ms/step - loss: 0.1289 - accuracy: 0.9599 - val_loss: 0.7312 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0803 - accuracy: 0.9677 - val_loss: 0.9732 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0681 - accuracy: 0.9717 - val_loss: 1.0260 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0578 - accuracy: 0.9755 - val_loss: 1.1908 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0498 - accuracy: 0.9752 - val_loss: 1.1593 - val_accuracy: 0.7690


In [71]:
#make predictions 
model4_pred_probs=model4.predict(val_sentences)
model4_pred_probs[:10]

array([[1.5994256e-03],
       [7.6979822e-01],
       [9.9992061e-01],
       [2.0103006e-01],
       [2.7527238e-05],
       [9.9952459e-01],
       [8.9723963e-01],
       [9.9997425e-01],
       [9.9994624e-01],
       [4.8044291e-01]], dtype=float32)

In [72]:
#convert pred probs to pred labels
model4_preds=tf.squeeze(tf.round(model4_pred_probs))
model4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [73]:
#calculate the result of our bidirectional model
model4_result=calculate_results(y_true=val_labels,y_pred=model4_preds)
model4_result

{'accuracy': 76.9028871391076,
 'f1': 0.7671875324347506,
 'precision': 0.7701356893081864,
 'recall': 0.7690288713910761}

In [74]:
model3_result

{'accuracy': 78.21522309711287,
 'f1': 0.7797935460636591,
 'precision': 0.7849731976577976,
 'recall': 0.7821522309711286}

it seems that bidirectional model performed worse than unidirectional 

#CNN for text  ( CONV1D)

In [77]:
#see how CONV1D works
embedding_test=embedding(text_vectorizer(["this is a test sentence "]))
conv1d=layers.Conv1D(filters=32,kernel_size=5,activation="relu",padding="valid")
conv1d_output=conv1d(embedding_test)
max_pool=layers.GlobalMaxPooling1D()# get the most important features
max_pool_out=max_pool(conv1d_output)
embedding_test.shape,conv1d_output.shape,max_pool_out.shape

 # kernel size = setting this to 5 means it'll look at 5 words at a time, 3 would mean 3 words at a time

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [87]:
#create CNN model with functional API

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Conv1D(64,kernel_size=5,activation="relu",padding="valid")(x)
x=layers.GlobalMaxPooling1D()(x)
#x=layers.Conv1D(64,5,activation="relu",padding="valid")(x)
#x=layers.GlobalMaxPooling1D()(x)
#x=layers.Conv1D(64,5,activation="relu",padding="valid")(x)
#x=layers.GlobalMaxPooling1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model5=tf.keras.Model(inputs,outputs,name="model5_with_CNN")

In [88]:
model5.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [84]:
model5.summary()

Model: "model5 with CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_5 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_3 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)             

In [89]:
history5=model5.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels), callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model5")])

Saving TensorBoard log files to: model_logs/model5/20220712-065405
Epoch 1/5
215/215 [==============================] - 3s 6ms/step - loss: 0.1292 - accuracy: 0.9507 - val_loss: 0.8730 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0866 - accuracy: 0.9656 - val_loss: 1.0501 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0689 - accuracy: 0.9723 - val_loss: 1.0596 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0592 - accuracy: 0.9758 - val_loss: 1.1491 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0539 - accuracy: 0.9777 - val_loss: 1.2376 - val_accuracy: 0.7664


In [90]:
# prediction
model5_pred_probs=model5.predict(val_sentences)
model5_pred_probs[:10]

array([[2.1703285e-01],
       [3.3694237e-01],
       [9.9990022e-01],
       [3.0398244e-02],
       [7.2901514e-08],
       [9.9633622e-01],
       [9.5756912e-01],
       [9.9993527e-01],
       [9.9999893e-01],
       [8.3306336e-01]], dtype=float32)

In [93]:
#convert model5 pred probs to labels
model5_preds=tf.squeeze(tf.round(model5_pred_probs))
model5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [98]:
model5_result=calculate_results(y_true=val_labels,y_pred=model5_preds)
model5_result ,"model 5 vs baseline model", baseline_result

({'accuracy': 76.64041994750657,
  'f1': 0.7642857598265804,
  'precision': 0.7679094748917975,
  'recall': 0.7664041994750657},
 'model 5 vs baseline model',
 {'accuracy': 79.26509186351706,
  'f1': 0.7862189758049549,
  'precision': 0.8111390004213173,
  'recall': 0.7926509186351706})

our CNN model still worse than base model 